In [33]:
import pandas as pd
import numpy as np
from espn_api.football import League
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

# Starting variables and data structures

In [36]:
# Access the league metadata variables
league_id = os.getenv("league_id")
espn_s2 = os.getenv("espn_s2")
swid = os.getenv("swid")

# Access team owners assign initials
team_1_name = os.getenv('team_1_name')
team_2_name = os.getenv('team_2_name')
team_3_name_1 = os.getenv('team_3_name_1')
team_3_name_2 = os.getenv('team_3_name_1')
team_4_name = os.getenv('team_4_name')
team_5_name = os.getenv('team_5_name')
team_6_name = os.getenv('team_6_name')
team_7_name = os.getenv('team_7_name')
team_8_name = os.getenv('team_8_name')
team_9_name = os.getenv('team_9_name')
team_10_name = os.getenv('team_10_name')
team_11_name = os.getenv('team_11_name')
team_12_name = os.getenv('team_12_name')
team_13_name = os.getenv('team_13_name')

team_1_initials = 'ZB'
team_2_initials = 'AL'
team_3_initials = 'BM'
team_4_initials = 'TB'
team_5_initials = 'FS'
team_6_initials = 'NH'
team_7_initials = 'GM'
team_8_initials = 'AM'
team_9_initials = 'DS'
team_10_initials = 'PS'
team_11_initials = 'CR'
team_12_initials = 'JK'
team_13_initials = 'DB'

In [3]:
# Update to be current season
max_year = 2025

In [4]:
league_2013 = League(league_id, 2013, espn_s2, swid, debug=False)
league_2014 = League(league_id, 2014, espn_s2, swid, debug=False)
league_2015 = League(league_id, 2015, espn_s2, swid, debug=False)
league_2016 = League(league_id, 2016, espn_s2, swid, debug=False)
league_2017 = League(league_id, 2017, espn_s2, swid, debug=False)
league_2018 = League(league_id, 2018, espn_s2, swid, debug=False)
league_2019 = League(league_id, 2019, espn_s2, swid, debug=False)
league_2020 = League(league_id, 2020, espn_s2, swid, debug=False)
league_2021 = League(league_id, 2021, espn_s2, swid, debug=False)
league_2022 = League(league_id, 2022, espn_s2, swid, debug=False)
league_2023 = League(league_id, 2023, espn_s2, swid, debug=False)
league_2024 = League(league_id, 2024, espn_s2, swid, debug=False)
league_2025 = League(league_id, 2025, espn_s2, swid, debug=False)

In [47]:
# Define the column names
columns = [
    "year",
    "week",
    #"home_owner",
    "home_score",
    "home_result",
    #"away_owner",
    "away_score",
    "away_result",
    "is_playoff",
    "playoff_type",
    "h2h_eligible"
]

# Create an empty DataFrame
league_history = pd.DataFrame(columns=columns)

# Functions

In [48]:
def get_weekly_matchups(lg, wk):
    """ Returns all matchups from a given week. """

    return lg.scoreboard(wk)

In [49]:
def get_h2h_eligibility(matchup_dict):
    """ Returns whether a matchup is h2h eligible or not. """

    if (matchup_dict['is_playoff'] == True) and matchup_dict['playoff_type'] != "WINNERS_BRACKET":
        return 0

    else:
        return 1

In [50]:
def create_matchup_dict(yr, wk, matchup):
    """ Creates a dictionary for a given matchup """

    try:
        matchup_dict = {
                    'year': yr,
                    'week': wk,
                    #'home_owner': matchup.home_team.owners[0]['firstName'],
                    'home_score': matchup.home_score,
                    'home_result': 0,
                    #'away_owner': matchup.away_team.owners[0]['firstName'],
                    'away_score': matchup.away_score,
                    'away_result': 0,
                    'is_playoff': matchup.is_playoff,
                    'playoff_type': matchup.matchup_type,
                    'h2h_eligible': 1
        }
    
        if matchup_dict['home_score'] > matchup_dict['away_score']:
            matchup_dict['home_result'] = 1
        elif matchup_dict['home_score'] < matchup_dict['away_score']:
            matchup_dict['away_result'] = 1
        else:
            matchup_dict['home_result'] = 0.5
            matchup_dict['away_result'] = 0.5


        matchup_dict['h2h_eligible'] = get_h2h_eligibility(matchup_dict)
    
        return matchup_dict

    except AttributeError:
        return None

    except Exception:
        return None

In [51]:
def create_season_history(yr, lg, lg_hist):
    """ Appends all matchups from a season into the league_history_data frame. """

    if lg.year == max_year:
        current_season_weeks = [i + 1 for i in range(lg.current_week - 1)]
    else:
        current_season_weeks = [i + 1 for i in range(lg.current_week)]

    for i in current_season_weeks:
        wk_scb = get_weekly_matchups(lg, i)

        for j in range(len(wk_scb)):
            game_result = create_matchup_dict(yr, i, wk_scb[j])
            
            if game_result != None:
                lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)

    return lg_hist

# Season Data Frame Creations

In [52]:
history_2024 = create_season_history(2024, league_2024, league_history)
history_2024

C:\Users\Paul\AppData\Local\Temp\ipykernel_5860\656331994.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)


,year,week,home_score,home_result,away_score,away_result,is_playoff,playoff_type,h2h_eligible
0,2024,1,87.16,0,88.92,1,False,NONE,1
1,2024,1,112.86,1,100.98,0,False,NONE,1
2,2024,1,138.88,1,137.58,0,False,NONE,1
3,2024,1,85.08,0,101.02,1,False,NONE,1
4,2024,1,80.14,0,84.86,1,False,NONE,1
...,...,...,...,...,...,...,...,...,...
98,2024,17,161.06,1,105.74,0,True,WINNERS_CONSOLATION_LADDER,0
99,2024,17,50.50,0,75.70,1,True,WINNERS_CONSOLATION_LADDER,0
100,2024,17,116.72,1,77.88,0,True,LOSERS_CONSOLATION_LADDER,0
101,2024,17,153.24,1,90.98,0,True,LOSERS_CONSOLATION_LADDER,0


In [53]:
history_2025 = create_season_history(2025, league_2025, league_history)
history_2025

C:\Users\Paul\AppData\Local\Temp\ipykernel_5860\656331994.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)


,year,week,home_score,home_result,away_score,away_result,is_playoff,playoff_type,h2h_eligible
0,2025,1,65.28,0,94.78,1,False,NONE,1
1,2025,1,110.42,1,93.42,0,False,NONE,1
2,2025,1,103.02,0,106.42,1,False,NONE,1
3,2025,1,80.36,0,101.42,1,False,NONE,1
4,2025,1,83.12,0,86.12,1,False,NONE,1
...,...,...,...,...,...,...,...,...,...
61,2025,11,79.10,1,72.42,0,False,NONE,1
62,2025,11,60.54,0,90.20,1,False,NONE,1
63,2025,11,116.22,0,119.98,1,False,NONE,1
64,2025,11,119.56,1,115.32,0,False,NONE,1


In [54]:
history_2013 = create_season_history(2013, league_2013, league_history)
history_2014 = create_season_history(2014, league_2014, league_history)
history_2015 = create_season_history(2015, league_2015, league_history)
history_2016 = create_season_history(2016, league_2016, league_history)
history_2017 = create_season_history(2017, league_2017, league_history)
history_2018 = create_season_history(2018, league_2018, league_history)
history_2019 = create_season_history(2019, league_2019, league_history)
history_2020 = create_season_history(2020, league_2020, league_history)
history_2021 = create_season_history(2021, league_2021, league_history)
history_2022 = create_season_history(2022, league_2022, league_history)
history_2023 = create_season_history(2023, league_2023, league_history)

C:\Users\Paul\AppData\Local\Temp\ipykernel_5860\656331994.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lg_hist = pd.concat([lg_hist, pd.DataFrame([game_result])], ignore_index = True)


In [55]:
league_matchup_history = [history_2013, history_2014, history_2015, history_2016, history_2017,
                          history_2018, history_2019, history_2020, history_2021, history_2022,
                          history_2023, history_2024, history_2025]  # your 13 dataframes
league_matchup_history = pd.concat(league_matchup_history, ignore_index=True)


In [56]:
league_matchup_history

,year,week,home_score,home_result,away_score,away_result,is_playoff,playoff_type,h2h_eligible
0,2013,1,148.00,1,89.00,0,False,NONE,1
1,2013,1,127.00,1,86.00,0,False,NONE,1
2,2013,1,114.00,1,68.00,0,False,NONE,1
3,2013,1,119.00,1,76.00,0,False,NONE,1
4,2013,1,116.00,1,54.00,0,False,NONE,1
...,...,...,...,...,...,...,...,...,...
1153,2025,11,79.10,1,72.42,0,False,NONE,1
1154,2025,11,60.54,0,90.20,1,False,NONE,1
1155,2025,11,116.22,0,119.98,1,False,NONE,1
1156,2025,11,119.56,1,115.32,0,False,NONE,1
